# "COVID-19 por comuna"
> Nuevos casos confirmados por comuna.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import numpy as np
import pandas as pd
import altair as alt

In [2]:
#hide
from IPython.display import display, Markdown, display_html, HTML

In [3]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(19)

In [4]:
#hide_input
display(Markdown(f"Última actualización: {update_date.strftime('%d/%m/%Y')}."))

Última actualización: 08/07/2020.

# Nuevos casos confirmados por comuna de la Región Metropolitana

In [5]:
#hide
county_cumulative_cases = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv")

In [6]:
#hide
county_cumulative_cases_RM = county_cumulative_cases.query("Region=='Metropolitana'").drop(
    columns=["Region", "Codigo region", "Poblacion", "Tasa", "Codigo comuna"])

In [7]:
#hide
county_cases_RM = county_cumulative_cases_RM.set_index("Comuna").T.diff().drop("2020-03-30")

In [8]:
#hide
county_cumulative_cases_RM.loc[107].tail(3)

2020-06-28    2778
2020-07-01    2827
2020-07-05    2962
Name: 107, dtype: object

In [9]:
#hide
county_cases_RM["Total"] = county_cases_RM.sum(axis=1)
county_cases_RM = county_cases_RM.T.apply(lambda x: x/x[-1]).T

In [10]:
#hide
start_date = county_cases_RM.reset_index()["index"].iloc[0]
end_date = county_cases_RM.reset_index()["index"].iloc[-1]

data = pd.DataFrame()
data['Fecha'] = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')
data = data.set_index("Fecha")

In [11]:
#hide
data[list(county_cases_RM.columns)] = county_cases_RM[list(county_cases_RM.columns)]

In [12]:
#hide
data = data.interpolate(method="linear")

In [13]:
#hide
last_known_proportion = data.iloc[-1]

In [14]:
#hide
last_known_date = county_cases_RM.index[-1]
last_known_date

'2020-07-05'

In [15]:
#hide
regional_cumulative_cases = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv')

In [16]:
#hide
regional_cumulative_cases_RM = regional_cumulative_cases.query("Region=='Metropolitana'").iloc[-1].drop("Region")
regional_cases_RM = regional_cumulative_cases_RM.diff().dropna()

In [17]:
#hide
n_missing_days = (pd.to_datetime(regional_cases_RM.index[-1])-pd.to_datetime(data.index[-1])).days

In [18]:
#hide
data = data.T

In [19]:
#hide
for i in np.arange(n_missing_days):
    idx = (pd.to_datetime(last_known_date)+pd.DateOffset(1+i)).strftime("%Y-%m-%d")
    print(idx)
    data[idx] = last_known_proportion

2020-07-06
2020-07-07
2020-07-08


In [20]:
#hide
data = data.T

In [21]:
#hide
data["Regional"] = regional_cases_RM

In [22]:
#hide
data = data.fillna(0)

In [23]:
#hide
outlier = data.loc["2020-06-17","Regional"]
outlier

32230.0

In [24]:
#hide
data.loc["2020-06-17","Regional"] = data.loc["2020-06-16","Regional"]

In [25]:
#hide
total = data.loc[:"2020-06-17", "Regional"].sum()
total

150904.0

In [26]:
#hide
proportion = data.loc[:"2020-06-17", "Regional"]/total
proportion.tail(2)

Fecha
2020-06-16    0.026653
2020-06-17    0.026653
Name: Regional, dtype: float64

In [27]:
#hide
data.loc[:"2020-06-17", "Regional"] = data.loc[:"2020-06-17", "Regional"] + (proportion*outlier).astype(int)

In [28]:
#hide
regional = data["Regional"].copy()

In [29]:
#hide
data = data.T.apply(lambda x: x[:-1]*x[-1]).T.astype(int)

In [30]:
#hide
data_long_form = data.drop(columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Casos confirmados")

In [31]:
#hide
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [32]:
#hide_input
input_dropdown = alt.binding_select(options=data_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='category20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(data_long_form).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="número de casos confirmados")),
    tooltip = ["Fecha", "Comuna", "Casos confirmados"],
    color = color
).properties(
    title="Nuevos casos confirmados por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19), [Ministerio de Salud](https://www.minsal.cl/)

* No hay datos diarios de casos confirmados por comuna. Estos son estimados a través de una interpolación lineal entre las proporciones observadas entre los dos informes más cercanos en el tiempo y aplicados a casos confirmados regionales.

# Total casos confirmados por comuna de la Región Metropolitana

In [33]:
#hide
data_long_form = data.cumsum().drop(columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Casos confirmados")

In [34]:
#hide_input
input_dropdown = alt.binding_select(options=data_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='category20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(data_long_form).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="número de casos confirmados")),
    tooltip = ["Fecha", "Comuna", "Casos confirmados"],
    color = color
).properties(
    title="Total casos confirmados por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

In [35]:
#hide
county_total_cases_RM = county_cumulative_cases_RM.set_index("Comuna").T

In [36]:
#hide
county_total_cases_RM["Total"] = county_total_cases_RM.sum(axis=1)
county_total_cases_RM = county_total_cases_RM.T.apply(lambda x: x/x[-1]).T

In [37]:
#hide
start_date = county_total_cases_RM.reset_index()["index"].iloc[0]
end_date = county_total_cases_RM.reset_index()["index"].iloc[-1]

In [38]:
#hide
data = pd.DataFrame()
data['Fecha'] = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')
data = data.set_index("Fecha")

In [39]:
#hide
data[list(county_total_cases_RM.columns)] = county_total_cases_RM[list(county_total_cases_RM.columns)]

In [40]:
#hide
data = data.interpolate(method="linear")

In [41]:
#hide
last_known_proportion = data.iloc[-1]

In [42]:
#hide
last_known_date = county_total_cases_RM.index[-1]
last_known_date

'2020-07-05'

In [43]:
#hide
#regional_cumulative_cases = pd.read_csv(
#    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv')

In [44]:
#hide
# regional_cumulative_cases_RM = regional_cumulative_cases.query("Region=='Metropolitana'").iloc[-1].drop("Region")

In [45]:
#hide
regional_cumulative_cases_RM = regional.cumsum()

In [46]:
#hide
n_missing_days = (pd.to_datetime(regional_cumulative_cases_RM.index[-1])-pd.to_datetime(data.index[-1])).days

In [47]:
#hide
data = data.T

In [48]:
#hide
for i in np.arange(n_missing_days):
    idx = (pd.to_datetime(last_known_date)+pd.DateOffset(1+i)).strftime("%Y-%m-%d")
    print(idx)
    data[idx] = last_known_proportion

2020-07-06
2020-07-07
2020-07-08


In [49]:
#hide
data = data.T

In [50]:
#hide
data["Regional"] = regional_cumulative_cases_RM

In [51]:
#hide
data = data.fillna(0)

In [52]:
#hide
data = data.T.apply(lambda x: x[:-1]*x[-1]).T.astype(int)

In [53]:
#hide
data_long_form = data.drop(columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Casos confirmados")

In [54]:
#hide
input_dropdown = alt.binding_select(options=data_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='category20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(data_long_form).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="total casos confirmados")),
    tooltip = ["Fecha", "Comuna", "Casos confirmados"],
    color = color
).properties(
    title="Total casos confirmados por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19), [Ministerio de Salud](https://www.minsal.cl/)

* No hay datos diarios de casos confirmados por comuna. Estos son estimados a través de una interpolación lineal entre las proporciones observadas entre los dos informes más cercanos en el tiempo y aplicados a casos confirmados regionales.

# Total fallecidos confirmados por comuna de la Región Metropolitana

In [66]:
#hide
county_deaths = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto38/CasosFallecidosPorComuna.csv")

In [67]:
#hide
county_deaths.T.tail(3)

,0,1,2,3,4,5,6,7,8,9,...,368,369,370,371,372,373,374,375,376,377
2020-06-28,22,0,0,0,0,22,25,2,2,3,...,0,0,0,22,0,0,0,0,0,22
2020-07-01,22,0,0,0,0,22,25,2,2,3,...,0,0,0,22,0,0,0,0,0,22
2020-07-05,23,0,0,0,0,23,30,2,2,3,...,0,0,0,23,0,0,0,0,0,23


In [57]:
#hide
last_date = county_deaths.T.iloc[-1].name
last_date

'2020-07-05'

In [58]:
#hide
county_deaths_RM = county_deaths.query(
    "Region=='Metropolitana'").drop(
    columns=["Region", "Codigo region", "Codigo comuna"]).set_index("Comuna")

In [59]:
#hide
county_deaths_RM = county_deaths_RM.rename(columns={last_date: "Fallecidos confirmados"})

In [60]:
#hide_input
bars = alt.Chart(county_deaths_RM.drop("Total").reset_index()).mark_bar(opacity=0.8, size=25).encode(
    x = alt.X('Fallecidos confirmados'),
    y = alt.Y('Comuna', sort='-x'),
    tooltip = ['Comuna', 'Poblacion', 'Fallecidos confirmados'],
    color = alt.Color('Comuna', scale=alt.Scale(scheme='tableau20'), legend=None),
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to the right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Fallecidos confirmados')
)

bars.properties(
    title = f"Total fallecidos confirmados por comuna de la R.M. al {pd.to_datetime(last_date).strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
) + text

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19), [Ministerio de Salud](https://www.minsal.cl/)

# Total fallecidos confirmados por 100.000 habitantes por comuna de la Región Metropolitana

In [61]:
#hide
county_deaths_RM["Tasa de mortalidad"] = \
np.round(100000*county_deaths_RM["Fallecidos confirmados"]/county_deaths_RM["Poblacion"], decimals=1)

In [62]:
#hide_input
bars = alt.Chart(county_deaths_RM.drop("Total").reset_index()).mark_bar(opacity=0.8, size=25).encode(
    x = alt.X('Tasa de mortalidad'),
    y = alt.Y('Comuna', sort='-x'),
    tooltip = ['Comuna', 'Poblacion', 'Fallecidos confirmados', 'Tasa de mortalidad'],
    color = alt.Color('Comuna', scale=alt.Scale(scheme='tableau20'), legend=None),
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to the right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Tasa de mortalidad')
)

bars.properties(
    title = f"Total fallecidos confirmados por 100.000 habitantes por comuna de la R.M. al {pd.to_datetime(last_date).strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
) + text

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19), [Ministerio de Salud](https://www.minsal.cl/)

# Letalidad por comuna de la Región Metropolitana

In [63]:
#hide
county_RM = county_cumulative_cases_RM[["Comuna", last_date]]
county_RM = county_RM.rename(columns={last_date: "Casos confirmados"}).set_index("Comuna")

In [64]:
#hide
county_RM["Fallecidos confirmados"] = county_deaths_RM["Fallecidos confirmados"]
county_RM["Letalidad"] = np.round(county_RM["Fallecidos confirmados"]/county_RM["Casos confirmados"], decimals=3)
county_RM["Texto"] = [f"{np.round(100*county_RM['Letalidad'].iloc[i], decimals=1)}%" for i in np.arange(len(county_RM))]

In [65]:
#hide_input
bars = alt.Chart(county_RM.reset_index()).mark_bar(opacity=0.8, size=25).encode(
    #x = alt.X('Letalidad'),
    x = alt.X('Letalidad', axis=alt.Axis(format='%', title='Letalidad (%)')),
    y = alt.Y('Comuna', sort='-x'),
    tooltip = ['Comuna', 'Casos confirmados', 'Fallecidos confirmados', 'Letalidad'],
    color = alt.Color('Comuna', scale=alt.Scale(scheme='tableau20'), legend=None),
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to the right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto'),
)

bars.properties(
    title = f"Letalidad por comuna de la R.M. al {pd.to_datetime(last_date).strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
) + text

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19), [Ministerio de Salud](https://www.minsal.cl/)

# Advertencia

El número de casos confirmados/informados o fallecimientos confirmados/por o con Covid-19 no representa exactamente el número de casos/contagios o fallecimientos con Covid-19 reales. Estos últimos valores no son conocidos por límites de testeo y por dificultades en la atribución de causas de fallecimiento. Además, los casos/fallecimientos confirmados per capita/por X habitantes, dan mayores valores a comunas con pocos habitantes, por lo que introducen sesgos en los datos.